# Now, lets focus on getting our data ready, so we could make embeddings. 
- Since we are predicting the 'next word' in a sentence. We do not have any default labels.
- We will be using our input text to create a label. While creating a dataset, during each iteration, the output for the previous iteration will be a part of the input in our current iteration.
- We will use a sliding-window approach to achive this. 

In [1]:
import tiktoken 


In [3]:
tokenizer = tiktoken.get_encoding("gpt2")

In [4]:
with open('dprpht.txt', 'r', encoding='utf-8') as f:
    raw_data = f.read()
token_ids = tokenizer.encode(raw_data)
print(len(token_ids))

23470


In [7]:
sample_enc = token_ids[50:]
# we will just skip first 50 tokenIds (this is just to remove the unnecessary data at the top of the file)

### Lets, see an example to see how we create inputs and output labels. 
- context_len is just the number of tokens we will see at a time.
- lets take context_len = 4, for example. 

In [8]:
context_len = 4 
x = sample_enc[:context_len]     #input text
y = sample_enc[1:context_len+1]  #corresponding output labels. In the 
print(f"{x}")
print(f"   {y}")

[743, 4866, 340, 11]
   [4866, 340, 11, 1577]


In [9]:
for i in range(1, context_len+1):
    context = sample_enc[:i]
    desired = sample_enc[i]
    print(context, "---->", desired)

[743] ----> 4866
[743, 4866] ----> 340
[743, 4866, 340] ----> 11
[743, 4866, 340, 11] ----> 1577


In [17]:
for i in range(1, context_len+1):
    context = sample_enc[:i]
    desired = sample_enc[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 may ---->  copy
 may copy ---->  it
 may copy it ----> ,
 may copy it, ---->  give


### Now that we know how our inputs and labels look, lets create a Dataset and a DataLoader using PyTorch
- A custom Dataset class must implement three functions: \__init\_\_, \__len\_\_ , and \__getitem\_\_.

In [18]:
import torch
from torch.utils.data import Dataset, DataLoader

In [30]:
#Implementing a custom Dataset Class
class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        # text : The document we want to use to create our dataset
        # tokenizer : BPE tokenizer that we imported from tiktoken
        # max_length : Context length (no. of token our model can see)
        # stride : The number of units our next input should be ahead of our current input 
        self.input_ids = []
        self.output_ids = []

        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i: i+max_length]
            output_chunk = token_ids[i+1: i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.output_ids.append(torch.tensor(output_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.output_ids[idx]
        
        

### At the heart of PyTorch data loading utility is the torch.utils.data.DataLoader class. It represents a Python iterable over a dataset, with support for

- map-style and iterable-style datasets,
- customizing data loading order,
- automatic batching,
- single- and multi-process data loading,
- automatic memory pinning.
- These options are configured by the constructor arguments of a DataLoader, which has signature:
```
DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, *, prefetch_factor=2,
           persistent_workers=False)
```

In [31]:
def create_dataloader_v1(text, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    #initialize tokenizer 
    tokenizer = tiktoken.get_encoding("gpt2")
    #create Dataset
    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers) 
    return dataloader

In [32]:
start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK THE PROPHET ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK THE PROPHET ***"

start_idx = raw_data.find(start_marker)
end_idx = raw_data.find(end_marker)

if start_idx == -1 or end_idx == -1:
    raise ValueError("Start or end index not found")
   
# Slice the content between markers
trimmed_raw_data = raw_data[start_idx + len(start_marker):end_idx]


In [33]:
dataloader = create_dataloader_v1(trimmed_raw_data, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)

In [34]:
print(first_batch)

[tensor([[  628,   628,   198, 10970]]), tensor([[  628,   198, 10970,  4810]])]


In [35]:
second_batch = next(data_iter)

In [36]:
second_batch

[tensor([[  628,   198, 10970,  4810]]),
 tensor([[  198, 10970,  4810,  3185]])]

In [38]:
dataloader = create_dataloader_v1(trimmed_raw_data, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  628,   628,   198, 10970],
        [ 4810,  3185,    39,  2767],
        [  198,   198,  3886,   509],
        [15668,   346, 12488,  2596],
        [  198,   198,  3791,  1971],
        [   25, 22044,   317,    13],
        [ 6102,   404,    69,   198],
        [  198,  1129,  1954,   198]])

Targets:
 tensor([[  628,   198, 10970,  4810],
        [ 3185,    39,  2767,   198],
        [  198,  3886,   509, 15668],
        [  346, 12488,  2596,   198],
        [  198,  3791,  1971,    25],
        [22044,   317,    13,  6102],
        [  404,    69,   198,   198],
        [ 1129,  1954,   198,   198]])
